In [1]:
#!pip3 install --upgrade pip
!pip install python-dotenv openai
!pip3 install pinecone-client

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached openai-1.42.0-py3-none-any.whl.metadata (22 kB)
  Using cached anyio-4.4.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached jiter-0.5.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (3.6 kB)
  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (125 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached certifi-2024.7.4-py3-none-any.whl.metadata (2.2 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.20.1-cp312-cp312-macosx_11_0_arm64.whl.met

In [2]:
# To check your SDK version
!pip show pinecone-client

Name: pinecone-client
Version: 5.0.1
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: /Users/it/Documents/prof_ratemyprofessor/venv/lib/python3.12/site-packages
Requires: certifi, pinecone-plugin-inference, pinecone-plugin-interface, tqdm, typing-extensions, urllib3
Required-by: 


In [4]:
# To use the chat completion endpoint for Pinecone Assistant with the Python SDK
pip install --upgrade pinecone-client pinecone-plugin-assistant

SyntaxError: invalid syntax (4074291565.py, line 2)

## Create requirements.txt

After installing the necessary libraries, you can use `pip-chill` to create a file called `requirements.txt` which helps the team install the required libraries to run this project on their local machine.

In [5]:
pip install pip-chill

  Using cached pip_chill-1.0.3-py2.py3-none-any.whl.metadata (6.6 kB)
Using cached pip_chill-1.0.3-py2.py3-none-any.whl (6.9 kB)
Note: you may need to restart the kernel to use updated packages.


Run the following command to generate the `requirements.txt` file:

In [6]:
pip-chill >requirements.txt


Usage:   
  /Users/it/Documents/prof_ratemyprofessor/venv/bin/python -m pip <command> [options]

no such option: -c
Note: you may need to restart the kernel to use updated packages.


Uncomment the following line to install the required libraries from the `requirements.txt` file:

In [ ]:
# pip install -r requirements.txt

In [75]:
from dotenv import load_dotenv
import os

from pinecone import Pinecone
from pinecone import ServerlessSpec

# load env file
load_dotenv()

True

In [76]:
# Initialize Pinecone
pc = Pinecone(api_key=os.getenv('PINECODE_API_KEY'))
pc

When creating an index, import the `ServerlessSpec` class from the `pinecone` module.
If you created the index directly from Pinecone, you can skip this step.

In [77]:
pc.create_index(
  name="rmp-ai",
  dimension=1536,
  metric="cosine",
  spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  )
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'b4208a6e5de9ed08c7c17ab303cff5c9', 'Date': 'Sat, 24 Aug 2024 07:02:29 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [72]:
import json

data = json.load(open('users.json'))
users = data["users"]
users[0]

{'name': 'Alice Johnson',
 'location': 'New York, NY',
 'tech-stack': [{'name': 'Python', 'years': 5},
  {'name': 'Django', 'years': 3},
  {'name': 'JavaScript', 'years': 4}],
 'interests': 'AI and Machine Learning',
 'goal': 'Develop AI-driven web applications',
 'idea': 'Create a platform for personalized AI tutors'}

## Embedding data to OpenAI
Here, we are going to convert our string data into a semantic vector representation. Basically, we are goingt to get some numerical representation of the text, which can be used for similarity searches. This is done using the Pinecone Text Embedding model.

In [80]:
from openai import OpenAI

# instantiate the OpenAI client
client = OpenAI()

processed_data = []

profiles = []
stacks = []
for user in users:
    users_tech = ""
    for stack in user["tech-stack"]:
        users_tech += stack["name"] + ": " + str(stack["years"]) + "years, "
    stacks.append(users_tech)
    profiles.append("techstacks: " + users_tech + "; interests: " + user["interests"] + "; goal: " + user["goal"] + "; idea: " + user["idea"])


for i, user in enumerate(users):
    response = client.embeddings.create(
        input= profiles[i],
        model="text-embedding-3-small", 
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": user["name"],
        "metadata": {
            "location": user["location"],
            "tech-stack": stacks[i],
            "interests": user["interests"],
            "goal": user["goal"],
            "idea": user["idea"],
        }
    })

In [81]:
processed_data[0]

{'values': [-0.0452359,
  -0.032378014,
  0.021334752,
  0.0109590115,
  0.037070103,
  -0.023965953,
  0.004575436,
  0.06812604,
  -0.019455325,
  0.040932655,
  0.037096027,
  -0.05381646,
  -0.031211473,
  -0.03235209,
  0.014179964,
  0.032922402,
  -0.028930236,
  -0.0062312772,
  0.007135347,
  -0.010395183,
  -0.010790511,
  -0.0037394145,
  0.02237168,
  0.023810413,
  0.05511262,
  0.000896779,
  0.023019757,
  0.0918198,
  -0.025936112,
  -0.013227288,
  0.020038595,
  -0.03069301,
  -0.034840714,
  0.011555244,
  0.0381848,
  -0.029085774,
  0.026648998,
  0.018340629,
  0.064859726,
  -0.009876721,
  0.025171379,
  -0.054594155,
  0.028048849,
  0.0075177136,
  -0.032118782,
  -0.010887723,
  -0.034425944,
  0.016720433,
  0.0006063587,
  0.049176216,
  -0.04873552,
  0.01127009,
  -0.039869804,
  0.050213143,
  -0.02667492,
  0.012715305,
  -0.021645831,
  0.029189467,
  -0.000921892,
  0.018807245,
  0.03185955,
  0.009286969,
  0.030407855,
  0.026389766,
  -0.004432859

## Vectorize data and index it in Pinecone

Here, we gonna upsert the processed data into the Pinecone index. This is similar to insert or update in a database.

In [82]:
index = pc.Index("rmp-ai")

index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 30}

To check the summary or description of our index. check if the `total_vector_count` corresponds to the number of reviews we have.

In [87]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 30}},
 'total_vector_count': 30}